# Text Generation using GPT-2 Model

This notebook demonstrates how to load and run a pre-trained GPT-2 model to generate text based on a user prompt. We'll use the `transformers` library from Hugging Face to load the model and handle the text generation process.


In [ ]:
%pip install transformers torch ipywidgets

### Install the necessary libraries

In this step, we install the `transformers` library, which provides the tools to load and use pre-trained models from Hugging Face, and `torch`, which is the PyTorch library used to run the model.


### Import necessary libraries from Hugging Face


In [2]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch

We are importing two important classes: `GPT2LMHeadModel` (for loading the GPT-2 model) and `GPT2Tokenizer` (for converting text to a format the model can understand). We also import `torch` to help manage the model and move it to GPU if available.


### Load the GPT-2 model and tokenizer


In [ ]:
tokenizer = GPT2Tokenizer.from_pretrained("gpt2-medium")
model = GPT2LMHeadModel.from_pretrained("gpt2-medium")
device = torch.device("cpu")
model.to(device)

Here, we load the pre-trained `gpt2-medium` model and tokenizer. This step downloads the model from Hugging Face's model hub if you don't have it locally.
This could take several minutes!


### Function to generate text based on input prompt


In [12]:
def generate_text(prompt, max_length=100, num_return_sequences=1, temperature=0.85):
    # Tokenize the input prompt
    input_ids = tokenizer.encode(prompt, return_tensors='pt').to(device)
    
    # Create an attention mask
    attention_mask = torch.ones(input_ids.shape, device=device)

    # Generate text
    with torch.no_grad():
        output = model.generate(
            input_ids,
            max_length=max_length,
            num_return_sequences=num_return_sequences,
            no_repeat_ngram_size=2,
            repetition_penalty=1.5,
            top_p=0.92,
            temperature=temperature,
            do_sample=True,
            top_k=50,
            attention_mask=attention_mask,  # Provide attention mask
            pad_token_id=tokenizer.eos_token_id  # Set pad token ID explicitly
        )
    
    # Decode and return the generated text
    generated_texts = [tokenizer.decode(output[i], skip_special_tokens=True) for i in range(num_return_sequences)]
    return generated_texts

In this function:

- We **tokenize** the input text (i.e., convert it into a format the model can process).
- We use the model's `generate()` function to produce text based on the prompt. We set parameters like `max_length` (the maximum number of tokens to generate) and `temperature` (which controls the creativity/randomness of the generation).
- The function returns the generated text, decoded back into human-readable format.


### Generate text from a prompt


In [13]:
prompt = input("Enter a prompt: ")
generated_texts = generate_text(prompt, max_length=100, num_return_sequences=1)

print("\nGenerated Text:\n")
for idx, text in enumerate(generated_texts):
    print(f"{idx+1}: {text}\n")


Generated Text:

1: Why did the chicken cross the road?
In order to avoid being hit by a car, he would stop at red lights. He also started driving faster when approaching his destination after picking up chickens from farms and markets where there were already cars parked behind him. His behavior was out of control but as long it wasn't for what is supposed in this country...



This code block allows you to input a text prompt. The function will generate text based on that input and display the result(s).


### Interactive Widgets (Optional)

To make the notebook more interactive, you can use widgets to adjust parameters like `max_length` and `temperature`. Below is an example of how you can add a slider for `max_length` and `temperature`:


In [ ]:
import ipywidgets as widgets
from IPython.display import display

prompt = widgets.Text(value='Enter your text here', description='Prompt:', disabled=False)
max_length = widgets.IntSlider(min=50, max=300, step=10, value=100, description='Max Length')
temperature = widgets.FloatSlider(min=0.1, max=1.5, step=0.1, value=0.85, description='Temperature')

display(prompt, max_length, temperature)

def on_button_clicked(b):
    generated_texts = generate_text(prompt.value, max_length=max_length.value, temperature=temperature.value)
    for idx, text in enumerate(generated_texts):
        print(f"{idx+1}: {text}\n")

button = widgets.Button(description="Generate Text")
button.on_click(on_button_clicked)
display(button)